In [1]:
import pandas as pd





In [2]:
# Load datasets
df_resumes = pd.read_csv("UpdatedResumeDataSet.csv")
df_jobs = pd.read_csv("job_title_des.csv")

In [3]:
from sentence_transformers import InputExample

train_examples = []

# Create positive pairs for the same category
for index, row in df_resumes.iterrows():
    # Match resume with job description of the same category
    job_match = df_jobs[df_jobs['Job Title'].str.contains(row['Category'], case=False, na=False)]
    for _, job_row in job_match.iterrows():
        train_examples.append(InputExample(
            texts=[row['Resume'], job_row['Job Description']],
            label=1.0
        ))

# Create negative pairs by shuffling
import random
for index, row in df_resumes.iterrows():
    random_job = df_jobs.sample(n=1).iloc[0]
    train_examples.append(InputExample(
        texts=[row['Resume'], random_job['Job Description']],
        label=0.0
    ))


C:\Users\David Yu\SkillgapAnalyis2\SkillGapAnalysis\Backend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader
from datasets import Dataset
from accelerate import Accelerator
import torch

# Load model
model = SentenceTransformer("all-mpnet-base-v2")

# Prepare data loader for fine-tuning
train_dataloader = DataLoader(train_examples, batch_size=16, shuffle=True)

# Cosine Similarity Loss for fine-tuning
train_loss = losses.CosineSimilarityLoss(model)


In [5]:
torch.cuda.empty_cache()


In [6]:
# Fine-tune the model
model.to('cuda')

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    show_progress_bar=True,
    use_amp = True
    )
# Save the fine-tuned model
model.save("fine_tuned_sentence_transformer2")


Step,Training Loss
500,0.020800
1000,0.002700
1500,0.002700
2000,0.001700
2500,0.001800
3000,0.002100
3500,0.001200
4000,0.001100
4500,0.000800
5000,0.001100
